In this notebook, I am using the global statistical values (mean, max, min etc), spectral density of the sensor signals and shorttime fourier transforms as the features. 
I am using the principal components (first 21 components that explain upto 99.9% of the variance)of the spectral density features to reduce overfitting. 
Finally I am training an XGB model with five fold validation(Ordering the data in increasing order of time to eruption so as to ensure a similar distribution in each of the folds)

I referred to the following notebooks to obtain ideas and some code.Thanks to the respective authors.
1. https://www.kaggle.com/soheild91/ingv-nn-xgb-baseline (Getting started)
2. https://www.kaggle.com/amanooo/ingv-volcanic-basic-solution-stft (STFT features)
3. https://www.kaggle.com/kylesnyder/ingv-spectral-density-w-randomforest (Spectral density features)

In [ ]:
import numpy as np
import pandas as pd

import scipy
from scipy import signal
import random
import pickle

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import xgboost

# Spectral density features

In [ ]:
# #spectral density along with statistical features
# input_df = pd.read_csv('../input/predict-volcanic-eruptions-ingv-oe/train.csv')
# # data = np.empty((input_df.shape[0],1410))#129 features per sensor 10 sensors =1290 + 120 stat ftrs(12*10)
# data = np.empty((input_df.shape[0],1290))
# time = np.empty((input_df.shape[0],1))

# for i,segment in enumerate(input_df['segment_id']):
#     temp = pd.read_csv(f'../input/predict-volcanic-eruptions-ingv-oe/train/{segment}.csv').fillna(0)
#     temp_arr = np.empty((0,))
#     for col in temp.columns:
#         freq,psd =signal.welch(temp[col],100)
#         temp_arr= np.concatenate((temp_arr,psd))  
# #     temp_arr = np.concatenate((temp_arr,temp.abs().mean().to_numpy(),
# #                                 temp.std().to_numpy(),
# #                                 temp.mean().to_numpy(),
# #                                 temp.var().to_numpy(),
# #                                 temp.min().to_numpy(),
# #                                 temp.max().to_numpy(),
# #                                 temp.median().to_numpy(),
# #                                 temp.quantile([0.1,0.25,0.5,0.75,0.9]).to_numpy().reshape(1,-1)[0]))
#     temp_arr = temp_arr.reshape((1,-1))
#     data[i,:] = temp_arr
#     time[i,0] = input_df.loc[i,'time_to_eruption']

In [ ]:
# sample_submission_df=pd.read_csv('../input/predict-volcanic-eruptions-ingv-oe/sample_submission.csv',nrows=5)
# #data_test=np.empty((sample_submission_df.shape[0],1410))
# data_test=np.empty((sample_submission_df.shape[0],1290))
# for i,segment in enumerate(sample_submission_df['segment_id']):
#     temp=pd.read_csv(f'../input/predict-volcanic-eruptions-ingv-oe/test/{segment}.csv').fillna(0)
#     temp_arr = np.empty((0,))
#     for col in temp.columns:
#         freq,psd =signal.welch(temp[col],100)
#         temp_arr= np.concatenate((temp_arr,psd))
# #     temp_arr = np.concatenate((temp_arr,temp.abs().mean().to_numpy(),
# #                                 temp.std().to_numpy(),
# #                                 temp.mean().to_numpy(),
# #                                 temp.var().to_numpy(),
# #                                 temp.min().to_numpy(),
# #                                 temp.max().to_numpy(),
# #                                 temp.median().to_numpy(),
# #                                 temp.quantile([0.1,0.25,0.5,0.75,0.9]).to_numpy().reshape(1,-1)[0]))
#     temp_arr = temp_arr.reshape((1,-1))
#     data_test[i,:] = temp_arr

In [ ]:
# train_df_sd = pd.DataFrame(data)
# test_df_sd = pd.DataFrame(data_test)

In [ ]:
# with open('train_df_sd.pkl', 'wb') as f:
#     pickle.dump(train_df_sd, f)
    
# with open('test_df_sd.pkl', 'wb') as f:
#     pickle.dump(test_df_sd, f)

# STFT Features

In [ ]:
# # STFT(Short Time Fourier Transform) Specifications
# fs = 100                # sampling frequency 
# #N = len(segment_df)     # data size
# N = 60000
# n = 256                 # FFT segment size
# max_f = 20              # ～20Hz

# delta_f = fs / n        # 0.39Hz
# delta_t = n / fs / 2    # 1.28s

In [ ]:
# #path ../input/predict-volcanic-eruptions-ingv-oe/test
# def make_features(input_df,path):
#     feature_set = []
#     for i,segment_id in enumerate(input_df['segment_id']):
#         temp=pd.read_csv(f'{path}/{segment_id}.csv')#.fillna(0)
#         segment = [segment_id]
#         for sensor in temp.columns:
#             x = temp[sensor][:N] 
#             if x.isna().sum()>1000:
#                 segment += ([np.NaN] * 10)
#                 continue
#             f, t, Z = scipy.signal.stft(x.fillna(0), fs = fs, window = 'hann', nperseg = n)
#             f = f[:round(max_f/delta_f)+1]
#             Z = np.abs(Z[:round(max_f/delta_f)+1]).T    # ～max_f, row:time,col:freq

#             th = Z.mean() * 1     ##########
#             Z_pow = Z.copy()
#             Z_pow[Z < th] = 0
#             Z_num = Z_pow.copy()
#             Z_num[Z >= th] = 1

#             Z_pow_sum = Z_pow.sum(axis = 0)
#             Z_num_sum = Z_num.sum(axis = 0)

#             A_pow = Z_pow_sum[round(10/delta_f):].sum()
#             A_num = Z_num_sum[round(10/delta_f):].sum()
#             BH_pow = Z_pow_sum[round(5/delta_f):round(8/delta_f)].sum()
#             BH_num = Z_num_sum[round(5/delta_f):round(8/delta_f)].sum()
#             BL_pow = Z_pow_sum[round(1.5/delta_f):round(2.5/delta_f)].sum()
#             BL_num = Z_num_sum[round(1.5/delta_f):round(2.5/delta_f)].sum()
#             C_pow = Z_pow_sum[round(0.6/delta_f):round(1.2/delta_f)].sum()
#             C_num = Z_num_sum[round(0.6/delta_f):round(1.2/delta_f)].sum()
#             D_pow = Z_pow_sum[round(2/delta_f):round(4/delta_f)].sum()
#             D_num = Z_num_sum[round(2/delta_f):round(4/delta_f)].sum()
#             segment += [A_pow, A_num, BH_pow, BH_num, BL_pow, BL_num, C_pow, C_num, D_pow, D_num]

#         feature_set.append(segment)
#     cols = ['segment_id']
#     for i in range(10):
#         for j in ['A_pow', 'A_num','BH_pow', 'BH_num','BL_pow', 'BL_num','C_pow', 'C_num','D_pow', 'D_num']:
#             cols += [f's{i+1}_{j}']
#     feature_df = pd.DataFrame(feature_set, columns = cols)
#     feature_df['segment_id'] = feature_df['segment_id'].astype('int')
#     return feature_df


In [ ]:
# input_df = pd.read_csv('../input/predict-volcanic-eruptions-ingv-oe/train.csv')
# path_train = "../input/predict-volcanic-eruptions-ingv-oe/train"
# train_df_stft = make_features(input_df,path_train)
# train_df_stft = pd.merge(input_df, train_df_stft, on = 'segment_id')
# #test
# sample_submission_df=pd.read_csv('../input/predict-volcanic-eruptions-ingv-oe/sample_submission.csv')
# path_test = "../input/predict-volcanic-eruptions-ingv-oe/test"
# test_df_stft = make_features(sample_submission_df,path_test)


In [ ]:
# with open('train_df_stft.pkl', 'wb') as f:
#     pickle.dump(train_df_stft, f)
    
# with open('test_df_stft.pkl', 'wb') as f:
#     pickle.dump(test_df_stft, f)

In [ ]:
train_df_stft = pd.read_pickle('../input/volcano-features/train_df_stft.pkl')
time = train_df_stft['time_to_eruption']
train_df_stft = train_df_stft.drop(['segment_id', 'time_to_eruption'], axis=1)

# train_df_sd = pd.read_pickle('../input/volcano-features/train_df_sd_only.pkl')
train_df_sd = pd.read_pickle('../input/volcano-features/train_df_sd.pkl')

test_df_stft = pd.read_pickle('../input/volcano-features/test_df_stft.pkl')
test_df_stft = test_df_stft.drop(['segment_id'],axis = 1)

# test_df_sd = pd.read_pickle('../input/volcano-features/test_df_sd_only.pkl')
test_df_sd = pd.read_pickle('../input/volcano-features/test_df_sd.pkl')

# PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.999,svd_solver="full") #all components that explain upto 99.9% of the variance
train_pca = pca.fit_transform(train_df_sd)
test_pca = pca.transform(test_df_sd)
train_pca = pd.DataFrame(train_pca)
test_pca = pd.DataFrame(test_pca)

In [ ]:
train_df = pd.concat([train_df_stft,train_pca] ,axis=1)
test_df = pd.concat([test_df_stft,test_pca],axis = 1)

# XGB with stratified k fold 

In [ ]:
input_df = pd.read_csv('../input/predict-volcanic-eruptions-ingv-oe/train.csv')
input_df = input_df.sort_values("time_to_eruption")
fold_list = [1,2,3,4,5]
folds = []
for i in range(int((input_df.shape[0]-1)/5)):
    random.shuffle(fold_list)
    folds.extend(fold_list)
folds = folds + [1] #adding a remaining solitary record to fold 1
input_df['fold'] = folds
#input_df.head(20)

In [ ]:
predictions = np.zeros(len(test_df))
for fold in range(1,6):
    train_index_list = input_df[input_df['fold'] != fold].index
    test_index_list = input_df[input_df['fold'] == fold].index

    X_train = train_df.iloc[train_index_list]
    y_train = time[train_index_list]
    X_val = train_df.iloc[test_index_list]
    y_val = time[test_index_list]

    model = xgboost.XGBRegressor(n_estimators=100000,tree_method='gpu_hist',max_depth=8,learning_rate=0.05,alpha=0.1,SUBSAMPLE=0.6)#,colsample_bytree=0.5)
    eval_set = [(X_val, y_val)]
    model.fit(X_train, y_train,early_stopping_rounds=5,eval_metric='mae', eval_set=eval_set, verbose=False)
    print(model.evals_result()['validation_0']['mae'][-5:])
    predictions += model.predict(test_df)
predictions = predictions/5

sample_submission_df1=pd.read_csv('../input/predict-volcanic-eruptions-ingv-oe/sample_submission.csv')
sample_submission_df1['time_to_eruption']=predictions
sample_submission_df1.to_csv('xgb_5fldst_ft_sdpca_stft.csv',index=False)